In [1]:
import os
import numpy
from datetime import datetime, timedelta

In [2]:
ws_folder = '/disk1/workspace/20240116'
MYD_RAA_infos_npy = os.path.join(ws_folder, 'analysis', 'MYD03_RAA-matched_infos_10km_land.npy')

In [3]:
MYD_matched_infos = numpy.load(MYD_RAA_infos_npy)

In [4]:
MYD_matched_infos[0]

array(['173.855', '-40.855', '201811220310', '57.54', '58.115', '251.73',
       '314.961', '43.9', '284.11'], dtype='<U12')

In [5]:
pixel_res = 0.01
ahi_llon = 85.0
ahi_llat = 60.0

MCD_landcover_npy = os.path.join(ws_folder, 'reference', 'MCD12Q1.006.LC.AHI01km.npy')
mcd_landcover = numpy.load(MCD_landcover_npy)

In [6]:
cloudmask_folder = '/disk4/AHI_CloudMask'

def is_clearsky_record(matched_info_item):    
    lon = float(matched_info_item[0][:-1])
    lat = float(matched_info_item[1][:-1])
    mxd_date = datetime.strptime(matched_info_item[2], "%Y%m%d%H%M")
    ahi_date = mxd_date
    if matched_info_item[2][-1:] =='5':
        ahi_date = mxd_date + timedelta(minutes=5)
    ahi_time_str = ahi_date.strftime("%Y%m%d%H%M")
    if ahi_time_str[-4:] == '0240':
        ahi_date_temp = ahi_date - timedelta(minutes=10)
        ahi_time_str = ahi_date_temp.strftime("%Y%m%d%H%M")

    ahi_cm_dat = os.path.join(cloudmask_folder, ahi_time_str[:6], 'AHIcm.v0.'+ahi_time_str+'.dat')
    cloudmask = None
    cloudmask_res = 0.02
    if not os.path.exists(ahi_cm_dat):
        print('no file:', ahi_cm_dat)
        cloudmask = numpy.zeros((6000, 6000))
    else:
        try:
            cloudmask = numpy.fromfile(ahi_cm_dat, dtype=numpy.float32)
            cloudmask[cloudmask < 0.95] = 0.   # cloud
            cloudmask[cloudmask >= 0.95] = 1.
            cloudmask = cloudmask.reshape(6000, 6000)   # 0.02°
        except Exception as e:
            print(e)
            cloudmask = numpy.zeros((6000, 6000))

    cm_lon_idx = int((lon-85.)/cloudmask_res)
    cm_lat_idx = int((60.-lat)/cloudmask_res)
    is_clearsky = int(cloudmask[cm_lat_idx][cm_lon_idx])
    
    return is_clearsky, ahi_time_str

In [7]:
filter_infos = []
ahi_time_list = []
for raa_matched_info in MYD_matched_infos:
    # within AU 30S region
    lon = round(int(float(raa_matched_info[0])*100)/100,2)
    lat = round(int(float(raa_matched_info[1])*100)/100,2)
    if lon > 110 and lon < 125 and lat > -38 and lat < -28:
        # landcover
        lon_idx = int((lon - ahi_llon)/pixel_res)
        lat_idx = int((ahi_llat - lat)/pixel_res)
        landcover_idx = int(mcd_landcover[lat_idx][lon_idx])
        if landcover_idx in [10, 2]:
            # between May to August
            month_str = raa_matched_info[2][4:6]
            if float(month_str) >= 5 and float(month_str) <= 8:
                # is clear-sky
                is_clearsky, ahi_time_str = is_clearsky_record(raa_matched_info)
                if is_clearsky:
                    matched_info = raa_matched_info[:]
                    matched_info[0] = str(lon)
                    matched_info[1] = str(lat)
                    filter_infos.append(matched_info)
                    ahi_time_list.append(ahi_time_str)

In [8]:
len(MYD_matched_infos)

247859

In [9]:
len(filter_infos)

325

In [12]:
numpy.save(os.path.join(ws_folder, 'process', 'MYD_RAA_AU_30S_5-8_clear.npy'), numpy.array(filter_infos))

In [10]:
ahi_ac_list = []
for filtered_item_idx in range(len(filter_infos)):
    filtered_item = filter_infos[filtered_item_idx]
    lon = float(filtered_item[0])
    lat = float(filtered_item[1])
    ahi_time = ahi_time_list[filtered_item_idx]
    # landcover
    lon_idx = int((lon - ahi_llon)/pixel_res)
    lat_idx = int((ahi_llat - lat)/pixel_res)
    landcover_idx = int(mcd_landcover[lat_idx][lon_idx])
    ahi_ac_info = [str(lon), str(lat), ahi_time, str(landcover_idx)]
    ahi_ac_list.append(ahi_ac_info)

In [13]:
numpy.save(os.path.join(ws_folder, 'process', 'MYD_RAA_AU_30S_5-8_clear_ahi_time.npy'), numpy.array(ahi_ac_list))

In [11]:
for ahi_ax in ahi_ac_list:
    print(ahi_ax)

['123.45', '-33.65', '201805020610', '10']
['120.35', '-33.45', '201808040620', '10']
['120.45', '-33.65', '201808040620', '10']
['120.75', '-33.65', '201808040620', '10']
['116.55', '-33.65', '201806060640', '10']
['116.65', '-33.65', '201806060640', '10']
['116.75', '-33.65', '201806060640', '10']
['118.85', '-32.95', '201805230630', '10']
['118.95', '-33.05', '201805230630', '10']
['119.15', '-33.25', '201805230630', '10']
['119.25', '-33.25', '201805230630', '10']
['119.25', '-33.45', '201805230630', '10']
['119.35', '-33.45', '201805230630', '10']
['119.45', '-33.45', '201805230630', '10']
['119.45', '-33.55', '201805230630', '10']
['119.25', '-33.65', '201805230630', '10']
['119.35', '-33.65', '201805230630', '10']
['119.55', '-33.65', '201805230630', '10']
['119.25', '-33.75', '201805230630', '10']
['119.35', '-33.75', '201805230630', '10']
['119.55', '-33.75', '201805230630', '10']
['115.45', '-33.85', '201806290650', '2']
['115.65', '-34.45', '201806290650', '2']
['115.75', '-